## CNN in Tensorflow

#### Dataset Used : MNIST

**Step 1** : Importing the dataset

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [13]:
import tensorflow as tf

# Set the paramters
learning_rate = 0.00001
epochs = 10
batch_size = 128

#Numbers of samples to be used for test set, decrease if memory constraints
test_valid_size = 256

#Network Paramters
n_classes = 10
dropout = 0.75

**Step 2** : Set the weights and biases

In [14]:
weights = {
    'wc1' : tf.Variable(tf.random_normal([5, 5, 1, 32])), #5x5 is the filter size, it has 1 chann
    'wc2' : tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1' : tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out' : tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bd1' : tf.Variable(tf.random_normal([1024])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

##### Explanation of weights and biases used

```
weights = {
    'wc1' :  #5x5 is the filter size, it has 1 channel as input is flattened and ouput has 32 nodes which will serve as input to next layer,
    max pooling will be applied and the final dimensions will be 14*14*32
    'wc2' : the filter is 5*5*32 as that is the no of input nodes, here output is 64 nodes, final dimensions will be 7*7*64,
    'wd1' : a fully connected, the input is basically flattened and output layer has 1024 nodes
    'out' : the input layer with 1024 is coonverted into output of n_classes
}

#for biases the dimensions are 1 x the no of weights in the next layer
biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bd1' : tf.Variable(tf.random_normal([1024])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}
```

In [15]:
#method that returns a convolutional layer
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    x = tf.nn.relu(x)
    return x

In [16]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x,
                         ksize=[1, k, k, 1],
                         strides=[1, k, k, 1],
                         padding='SAME')

In [17]:
def conv_net(x, weights, biases, dropout):
    #Layer 1 28*28*1 -> 28*28*32 -> 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1)
    
    #Layer 2 14*14*32 -> 14*14*64 -> 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2)
    
    #Layer 3 7*7*64 -> (7*7*64) * 1024
    #reshapes the previous layer into [inferred, 7*7*64] dimensions
    #to be done when being multiplied with weights according to xW + b linear layer eq.
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    #Output Layer samples*1024 -> samples * n_classes
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

**Final Step** : Run the tensorflow session

In [20]:
#create the graph
x = tf.placeholder(tf.float32, [None, 28, 28, 1]) #grayscale image
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

logits = conv_net(x, weights, biases, keep_prob)

#Define loss and optomizer
cost = tf.reduce_mean(\
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
            .minimize(cost)


#calculate accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

#Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    #train the network
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x : batch_x,
                y : batch_y,
                keep_prob : dropout
            })
            
            
            
             #calculate accuracy on test_set
            loss = sess.run(cost, feed_dict={
                        x: batch_x,
                        y: batch_y,
                        keep_prob: 1.
             })
            
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.
            })
            
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

        
    
    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))
    

Epoch  1, Batch   1 -Loss: 49573.3594 Validation Accuracy: 0.097656
Epoch  1, Batch   2 -Loss: 39083.6992 Validation Accuracy: 0.105469
Epoch  1, Batch   3 -Loss: 35705.0625 Validation Accuracy: 0.089844
Epoch  1, Batch   4 -Loss: 28345.3145 Validation Accuracy: 0.093750
Epoch  1, Batch   5 -Loss: 31685.3184 Validation Accuracy: 0.078125
Epoch  1, Batch   6 -Loss: 29377.2207 Validation Accuracy: 0.070312
Epoch  1, Batch   7 -Loss: 27555.9297 Validation Accuracy: 0.074219
Epoch  1, Batch   8 -Loss: 27647.4688 Validation Accuracy: 0.089844
Epoch  1, Batch   9 -Loss: 23650.2012 Validation Accuracy: 0.082031
Epoch  1, Batch  10 -Loss: 20867.0645 Validation Accuracy: 0.093750
Epoch  1, Batch  11 -Loss: 21004.8438 Validation Accuracy: 0.101562
Epoch  1, Batch  12 -Loss: 20914.6328 Validation Accuracy: 0.113281
Epoch  1, Batch  13 -Loss: 19889.5098 Validation Accuracy: 0.128906
Epoch  1, Batch  14 -Loss: 22478.2598 Validation Accuracy: 0.128906
Epoch  1, Batch  15 -Loss: 18503.6328 Validation

Epoch  1, Batch 122 -Loss:  3929.8606 Validation Accuracy: 0.574219
Epoch  1, Batch 123 -Loss:  4007.4536 Validation Accuracy: 0.582031
Epoch  1, Batch 124 -Loss:  5313.0254 Validation Accuracy: 0.585938
Epoch  1, Batch 125 -Loss:  3564.2075 Validation Accuracy: 0.589844
Epoch  1, Batch 126 -Loss:  3849.2915 Validation Accuracy: 0.585938
Epoch  1, Batch 127 -Loss:  4970.3726 Validation Accuracy: 0.582031
Epoch  1, Batch 128 -Loss:  5465.7559 Validation Accuracy: 0.578125
Epoch  1, Batch 129 -Loss:  4195.0674 Validation Accuracy: 0.566406
Epoch  1, Batch 130 -Loss:  4497.9404 Validation Accuracy: 0.566406
Epoch  1, Batch 131 -Loss:  2840.9028 Validation Accuracy: 0.582031
Epoch  1, Batch 132 -Loss:  2816.2219 Validation Accuracy: 0.589844
Epoch  1, Batch 133 -Loss:  4290.4297 Validation Accuracy: 0.589844
Epoch  1, Batch 134 -Loss:  5244.5630 Validation Accuracy: 0.589844
Epoch  1, Batch 135 -Loss:  3674.8999 Validation Accuracy: 0.589844
Epoch  1, Batch 136 -Loss:  4576.9648 Validation

Epoch  1, Batch 243 -Loss:  2585.2854 Validation Accuracy: 0.710938
Epoch  1, Batch 244 -Loss:  2020.7151 Validation Accuracy: 0.710938
Epoch  1, Batch 245 -Loss:  2923.3750 Validation Accuracy: 0.707031
Epoch  1, Batch 246 -Loss:  2610.3108 Validation Accuracy: 0.710938
Epoch  1, Batch 247 -Loss:  1928.7980 Validation Accuracy: 0.710938
Epoch  1, Batch 248 -Loss:  2015.4922 Validation Accuracy: 0.714844
Epoch  1, Batch 249 -Loss:  1908.8298 Validation Accuracy: 0.710938
Epoch  1, Batch 250 -Loss:  2201.9292 Validation Accuracy: 0.707031
Epoch  1, Batch 251 -Loss:  1801.9857 Validation Accuracy: 0.718750
Epoch  1, Batch 252 -Loss:  2600.4636 Validation Accuracy: 0.710938
Epoch  1, Batch 253 -Loss:  2337.3672 Validation Accuracy: 0.707031
Epoch  1, Batch 254 -Loss:  2263.9941 Validation Accuracy: 0.707031
Epoch  1, Batch 255 -Loss:  2420.8574 Validation Accuracy: 0.707031
Epoch  1, Batch 256 -Loss:  1935.0342 Validation Accuracy: 0.707031
Epoch  1, Batch 257 -Loss:  2005.5951 Validation

Epoch  1, Batch 364 -Loss:  1356.2314 Validation Accuracy: 0.742188
Epoch  1, Batch 365 -Loss:  1801.3159 Validation Accuracy: 0.742188
Epoch  1, Batch 366 -Loss:  1318.9626 Validation Accuracy: 0.742188
Epoch  1, Batch 367 -Loss:  1412.7629 Validation Accuracy: 0.742188
Epoch  1, Batch 368 -Loss:  2394.5518 Validation Accuracy: 0.742188
Epoch  1, Batch 369 -Loss:  1800.5590 Validation Accuracy: 0.742188
Epoch  1, Batch 370 -Loss:  1497.4579 Validation Accuracy: 0.742188
Epoch  1, Batch 371 -Loss:  1028.5015 Validation Accuracy: 0.750000
Epoch  1, Batch 372 -Loss:  1300.9290 Validation Accuracy: 0.746094
Epoch  1, Batch 373 -Loss:  2037.1327 Validation Accuracy: 0.746094
Epoch  1, Batch 374 -Loss:  1297.1018 Validation Accuracy: 0.742188
Epoch  1, Batch 375 -Loss:  1538.0391 Validation Accuracy: 0.750000
Epoch  1, Batch 376 -Loss:  1405.5271 Validation Accuracy: 0.742188
Epoch  1, Batch 377 -Loss:  1402.2844 Validation Accuracy: 0.746094
Epoch  1, Batch 378 -Loss:  1299.4277 Validation

Epoch  2, Batch  56 -Loss:  1027.9017 Validation Accuracy: 0.761719
Epoch  2, Batch  57 -Loss:  1310.6626 Validation Accuracy: 0.761719
Epoch  2, Batch  58 -Loss:   740.9974 Validation Accuracy: 0.765625
Epoch  2, Batch  59 -Loss:   908.3425 Validation Accuracy: 0.765625
Epoch  2, Batch  60 -Loss:  1237.1417 Validation Accuracy: 0.761719
Epoch  2, Batch  61 -Loss:  1155.4546 Validation Accuracy: 0.761719
Epoch  2, Batch  62 -Loss:   891.3889 Validation Accuracy: 0.773438
Epoch  2, Batch  63 -Loss:  1072.7842 Validation Accuracy: 0.777344
Epoch  2, Batch  64 -Loss:  1037.7677 Validation Accuracy: 0.777344
Epoch  2, Batch  65 -Loss:  1612.1025 Validation Accuracy: 0.777344
Epoch  2, Batch  66 -Loss:   997.0343 Validation Accuracy: 0.777344
Epoch  2, Batch  67 -Loss:  1521.9752 Validation Accuracy: 0.781250
Epoch  2, Batch  68 -Loss:  1304.5292 Validation Accuracy: 0.781250
Epoch  2, Batch  69 -Loss:  1501.8528 Validation Accuracy: 0.773438
Epoch  2, Batch  70 -Loss:  1133.7535 Validation

...

Epoch  9, Batch 320 -Loss:   247.5093 Validation Accuracy: 0.812500
Epoch  9, Batch 321 -Loss:   198.8369 Validation Accuracy: 0.812500
Epoch  9, Batch 322 -Loss:   187.4695 Validation Accuracy: 0.812500
Epoch  9, Batch 323 -Loss:   246.1872 Validation Accuracy: 0.812500
Epoch  9, Batch 324 -Loss:   297.4856 Validation Accuracy: 0.812500
Epoch  9, Batch 325 -Loss:   229.3408 Validation Accuracy: 0.812500
Epoch  9, Batch 326 -Loss:   297.9776 Validation Accuracy: 0.812500
Epoch  9, Batch 327 -Loss:   236.5044 Validation Accuracy: 0.812500
Epoch  9, Batch 328 -Loss:   303.6432 Validation Accuracy: 0.816406
Epoch  9, Batch 329 -Loss:   242.5052 Validation Accuracy: 0.816406
Epoch  9, Batch 330 -Loss:   265.4856 Validation Accuracy: 0.820312
Epoch  9, Batch 331 -Loss:   251.4549 Validation Accuracy: 0.820312
Epoch  9, Batch 332 -Loss:   352.1300 Validation Accuracy: 0.824219
Epoch  9, Batch 333 -Loss:   301.3047 Validation Accuracy: 0.816406
Epoch  9, Batch 334 -Loss:   176.1465 Validation

Epoch 10, Batch  12 -Loss:   207.3201 Validation Accuracy: 0.820312
Epoch 10, Batch  13 -Loss:   346.2606 Validation Accuracy: 0.820312
Epoch 10, Batch  14 -Loss:   170.7537 Validation Accuracy: 0.820312
Epoch 10, Batch  15 -Loss:   221.8974 Validation Accuracy: 0.820312
Epoch 10, Batch  16 -Loss:   214.2928 Validation Accuracy: 0.820312
Epoch 10, Batch  17 -Loss:   240.3800 Validation Accuracy: 0.820312
Epoch 10, Batch  18 -Loss:   273.3244 Validation Accuracy: 0.820312
Epoch 10, Batch  19 -Loss:   286.6332 Validation Accuracy: 0.820312
Epoch 10, Batch  20 -Loss:   270.1930 Validation Accuracy: 0.816406
Epoch 10, Batch  21 -Loss:   229.0675 Validation Accuracy: 0.824219
Epoch 10, Batch  22 -Loss:   336.3349 Validation Accuracy: 0.820312
Epoch 10, Batch  23 -Loss:   334.5071 Validation Accuracy: 0.820312
Epoch 10, Batch  24 -Loss:   265.2504 Validation Accuracy: 0.820312
Epoch 10, Batch  25 -Loss:   238.5316 Validation Accuracy: 0.816406
Epoch 10, Batch  26 -Loss:   190.1109 Validation

Epoch 10, Batch 133 -Loss:   251.8971 Validation Accuracy: 0.820312
Epoch 10, Batch 134 -Loss:    96.1531 Validation Accuracy: 0.820312
Epoch 10, Batch 135 -Loss:   208.6893 Validation Accuracy: 0.816406
Epoch 10, Batch 136 -Loss:    77.4376 Validation Accuracy: 0.816406
Epoch 10, Batch 137 -Loss:   221.0487 Validation Accuracy: 0.816406
Epoch 10, Batch 138 -Loss:   257.4067 Validation Accuracy: 0.816406
Epoch 10, Batch 139 -Loss:   320.0653 Validation Accuracy: 0.816406
Epoch 10, Batch 140 -Loss:   236.0312 Validation Accuracy: 0.816406
Epoch 10, Batch 141 -Loss:   282.0858 Validation Accuracy: 0.820312
Epoch 10, Batch 142 -Loss:   226.7282 Validation Accuracy: 0.816406
Epoch 10, Batch 143 -Loss:   239.3403 Validation Accuracy: 0.816406
Epoch 10, Batch 144 -Loss:   235.6136 Validation Accuracy: 0.816406
Epoch 10, Batch 145 -Loss:   189.8331 Validation Accuracy: 0.816406
Epoch 10, Batch 146 -Loss:   252.1006 Validation Accuracy: 0.816406
Epoch 10, Batch 147 -Loss:   275.7523 Validation

Epoch 10, Batch 254 -Loss:   147.5591 Validation Accuracy: 0.820312
Epoch 10, Batch 255 -Loss:   161.8682 Validation Accuracy: 0.820312
Epoch 10, Batch 256 -Loss:   266.8664 Validation Accuracy: 0.820312
Epoch 10, Batch 257 -Loss:   215.0800 Validation Accuracy: 0.820312
Epoch 10, Batch 258 -Loss:   267.6015 Validation Accuracy: 0.816406
Epoch 10, Batch 259 -Loss:   243.3584 Validation Accuracy: 0.816406
Epoch 10, Batch 260 -Loss:   202.9533 Validation Accuracy: 0.816406
Epoch 10, Batch 261 -Loss:   193.2677 Validation Accuracy: 0.816406
Epoch 10, Batch 262 -Loss:   213.7025 Validation Accuracy: 0.808594
Epoch 10, Batch 263 -Loss:   174.6550 Validation Accuracy: 0.816406
Epoch 10, Batch 264 -Loss:   280.7497 Validation Accuracy: 0.808594
Epoch 10, Batch 265 -Loss:   185.5167 Validation Accuracy: 0.812500
Epoch 10, Batch 266 -Loss:   187.0842 Validation Accuracy: 0.812500
Epoch 10, Batch 267 -Loss:   295.4244 Validation Accuracy: 0.812500
Epoch 10, Batch 268 -Loss:   176.1761 Validation

Epoch 10, Batch 375 -Loss:   161.9720 Validation Accuracy: 0.808594
Epoch 10, Batch 376 -Loss:   227.8263 Validation Accuracy: 0.812500
Epoch 10, Batch 377 -Loss:   322.8777 Validation Accuracy: 0.804688
Epoch 10, Batch 378 -Loss:   158.7786 Validation Accuracy: 0.808594
Epoch 10, Batch 379 -Loss:   209.5143 Validation Accuracy: 0.812500
Epoch 10, Batch 380 -Loss:   200.6986 Validation Accuracy: 0.808594
Epoch 10, Batch 381 -Loss:   233.4593 Validation Accuracy: 0.808594
Epoch 10, Batch 382 -Loss:   128.1455 Validation Accuracy: 0.808594
Epoch 10, Batch 383 -Loss:   117.4861 Validation Accuracy: 0.808594
Epoch 10, Batch 384 -Loss:   184.4926 Validation Accuracy: 0.816406
Epoch 10, Batch 385 -Loss:   247.9934 Validation Accuracy: 0.808594
Epoch 10, Batch 386 -Loss:   220.1341 Validation Accuracy: 0.808594
Epoch 10, Batch 387 -Loss:   217.5045 Validation Accuracy: 0.816406
Epoch 10, Batch 388 -Loss:   286.8557 Validation Accuracy: 0.816406
Epoch 10, Batch 389 -Loss:   273.1269 Validation